In [1]:
import numpy as np
import h5py

from keras.models import Sequential
from keras.layers import Embedding, Dropout, Dense, Activation
from keras.layers import LSTM, Bidirectional, Merge, RepeatVector

Using TensorFlow backend.


In [2]:
with h5py.File('context.h5', 'r') as hf:
    context_array = hf['context'][:]
with h5py.File('questions.h5', 'r') as hf:
    question_array = hf['questions'][:]
with h5py.File('answers.h5', 'r') as hf:
    answer_array = hf['answers'][:]
with h5py.File('begin.h5', 'r') as hf:
    begin_span = hf['begin'][:]
with h5py.File('end.h5', 'r') as hf:
    end_span = hf['end'][:]

In [3]:
print context_array.shape
print question_array.shape
print answer_array.shape
print begin_span.shape
print end_span.shape

(87599, 1000)
(87599, 100)
(87599, 50)
(87599,)
(87599,)


In [4]:
vocab_size = 37
embedding_vector_length = 128
max_span = 1000
batch = 512

In [5]:
context_model = Sequential()
context_model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_vector_length, input_length=1000))
context_model.add(Bidirectional(LSTM(256, implementation=2), merge_mode='mul'))
context_model.add(Dropout(0.5))
context_model.add(RepeatVector(10))

In [6]:
ques_model = Sequential()
ques_model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_vector_length, input_length=100))
ques_model.add(Bidirectional(LSTM(256, implementation=2), merge_mode='mul'))
ques_model.add(Dropout(0.5))
ques_model.add(RepeatVector(10))

In [7]:
# Merge layers
model = Sequential()
model.add(Merge([context_model, ques_model], mode='mul'))
model.add(Bidirectional(LSTM(256, implementation=2), merge_mode='mul'))
model.add(Dropout(0.5))
model.add(Dense(max_span))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
             metrics=['accuracy'])
model.summary()

model_history = model.fit([context_array, question_array], begin_span,
                         verbose=2, batch_size=batch, epochs=10)

/home/blank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 10, 256)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               1050624   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              257000    
_________________________________________________________________
activation_1 (Activation)    (None, 1000)              0         
Total params: 2,894,056.0
Trainable params: 2,894,056.0
Non-trainable params: 0.0
_________________________________________________________________
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[1024,1024]
	 [[Node: bidirectional_1/while/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](bidirectional_1/while/mul, bidirectional_1/while/MatMul/Enter)]]
	 [[Node: mul_1/_79 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_9749_mul_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'bidirectional_1/while/MatMul', defined at:
  File "/home/blank/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/blank/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/blank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance